In [16]:
from dataset import Dataset
from classificadores import KNN, DMC, PerceptronSimples

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from typing import Union

In [13]:
# Conjunto de dados Câncer de Mama =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\wdbc.data", 
    label_column = 1,
    delimiter = ",",  
    column_names = ["ID", "Diagnosis", "radius1", "texture1", "perimeter1", "area1", "smoothness1", "compactness1", "concavity1", "concave_points1", "symmetry1", "fractal_dimension1", "radius2", "texture2", "perimeter2", "area2", "smoothness2", "compactness2", "concavity2", "concave_points2", "symmetry2", "fractal_dimension2", "radius3", "texture3", "perimeter3", "area3", "smoothness3", "compactness3", "concavity3", "concave_points3", "symmetry3", "fractal_dimension3"]
).ensure_numeric_labels().remove_features(["ID"]).normalize()

print(dataset)
dataset._label_index_to_name

Dataset(instâncias=569, features=30, classes=2)


{0: 'M', 1: 'B'}

In [17]:
# Conjunto de dados Coluna Vertebral =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\column_3C.dat", 
    label_column = -1,
    delimiter = " ",  
    column_names = ["pelvic incidence", "pelvic tilt", "lumbar lordosis angle", "sacral slope", "pelvic radius", "degree spondylolisthesis", "class"]
).ensure_numeric_labels().normalize()

print(dataset)
dataset._label_index_to_name

Dataset(instâncias=310, features=6, classes=3)


{0: 'DH', 1: 'SL', 2: 'NO'}

In [18]:
dataset.data

,pelvic incidence,pelvic tilt,lumbar lordosis angle,sacral slope,pelvic radius,degree spondylolisthesis,class
0,-0.288580,0.039657,-0.541614,-0.498242,-0.385095,-0.949674,0.0
1,-0.750965,-0.406574,-0.802756,-0.710716,-0.046564,-0.927281,0.0
2,-0.176698,0.027867,-0.354036,-0.384786,-0.227659,-0.964944,0.0
3,-0.167631,0.114684,-0.457491,-0.421247,-0.316271,-0.896322,0.0
4,-0.545525,-0.421222,-0.743691,-0.506015,-0.180772,-0.911639,0.0
...,...,...,...,...,...,...,...
305,-0.580440,-0.279385,-0.606229,-0.612808,0.018819,-0.968296,2.0
306,-0.463927,-0.025723,-0.727582,-0.632612,-0.047424,-0.950466,2.0
307,-0.319059,0.044659,-0.424199,-0.530261,0.195612,-0.961127,2.0
308,-0.631559,-0.455520,-0.506354,-0.570794,0.042478,-0.947533,2.0


In [19]:
dataset.determination_matrix()

,pelvic incidence,pelvic tilt,lumbar lordosis angle,sacral slope,pelvic radius,degree spondylolisthesis,class
pelvic incidence,1.000000,0.395875,0.514499,0.664158,0.061248,0.407979,0.000845
pelvic tilt,0.395875,1.000000,0.187282,0.003885,0.001067,0.158277,0.044820
lumbar lordosis angle,0.514499,0.187282,1.000000,0.358069,0.006459,0.284798,0.001351
sacral slope,0.664158,0.003885,0.358069,1.000000,0.117064,0.274127,0.014523
pelvic radius,0.061248,0.001067,0.006459,0.117064,1.000000,0.000680,0.055102
degree spondylolisthesis,0.407979,0.158277,0.284798,0.274127,0.000680,1.000000,0.014286
class,0.000845,0.044820,0.001351,0.014523,0.055102,0.014286,1.000000


In [20]:
dataset.vectorize_labels()
dataset.label_encodings

{np.float64(0.0): array([ 1., -1., -1.]),
 np.float64(1.0): array([-1.,  1., -1.]),
 np.float64(2.0): array([-1., -1.,  1.])}

In [21]:
# Separa o conjunto de dados em treinamento e teste
train_dataset, test_dataset = dataset.split()
train_dataset : Dataset 
test_dataset : Dataset

In [26]:
class MLP:
    def __init__( self, train_dataset : Dataset, q : int = 3):
        self.train_dataset = train_dataset
        
        self.q = q                                          # Número de neurônios ocultos
        self.m = self.train_dataset.class_count             # Número de neurônios de saída
        self.p = self.train_dataset.features_count + 1      # Número de entradas da rede

        # Função de ativação usada
        self.activation = lambda x: np.tanh(x)
        self.ddx_activation = lambda x: 1 - self.activation(x) ** 2

        # Inicializa o vetor de pesos dos neurônios ocultos
        self.W = np.random.normal( size = (self.q, self.p) )     # (q, p)

        # Inicializa o vetor de pesos dos neurônios de saída
        self.M = np.random.normal( size = (self.m, self.q+1) )   # (m, q+1)
    
    def train( self, max_epocas : int = 1000, *, eta : float = 0.1, reset_weights = False, verbose = True ):
        # Reseta os pesos
        if reset_weights:
            self.W = np.random.normal( size = (self.q, self.p) )
            self.M = np.random.normal( size = (self.m, self.q+1) )
        
        # Percorre um número qualquer de épocas
        for epoca in range( max_epocas ):
            total_erros = 0

            # Para cada época, embaralha o conjunto de treinamento
            shuffled_dataset = self.train_dataset.shuffle()

            # Percorre os exemplos do conjunto de treinamento
            for index, *features, classe in shuffled_dataset:
                # Vetor que representa a classe 
                real_output = self.train_dataset.encode_label( classe )

                # Monta o vetor de entrada
                X_bias = np.r_[features, -1]

                # Sentido direto - cálculo da ativação e a saída de cada camada
                U = self.W @ X_bias         # Ativação de cada neurônio oculto
                Y = self.activation( U )    # Saída dos neurônios ocultos

                Z = np.r_[ Y, -1 ]          # Prepara as entradas para os neurônios de saída

                A = self.M @ Z              # Ativação dos neurônio de saída
                O = self.activation ( A )   # Saída da camada de saída

                # Sentido inverso - atualização dos pesos das camadas
                err = real_output - O 

                # Verifica se houve erro
                if np.any(err != 0):
                    # atualiza os pesos da camada de saída
                    delta_output = err * self.ddx_activation( A )           # (m x 1)
                    delta_weights_out = eta * np.outer( delta_output, Z )   # (m x (q+1))
                    self.M = self.M + delta_weights_out

                    # calcula os erros retropagados para a camada oculta oculto
                    output_weights_no_bias_T = self.M[:, :-1].T                     # (q x m)
                    backpropagated_error = output_weights_no_bias_T @ delta_output  # (q x 1)

                    # atualiza os pesos da camada oculta
                    delta_hidden = backpropagated_error * self.ddx_activation(U)    # (q x 1)
                    delta_weights_hidden = eta * np.outer( delta_hidden, X_bias )   # (q x p)
                    self.W = self.W + delta_weights_hidden

                # Verifica se a previsão seria acertada ou não
                if np.argmax(O) != np.argmax(real_output):
                    total_erros += 1

            # Exibe uma mensagem de log a cada 5% do número máximo de épocas
            if verbose and not (epoca% round(max_epocas*0.05)):
                print(f"Época {epoca}: {total_erros} erros.")
    
    def predict( self, features : np.ndarray ) -> Union[float, int]:
        # Monta o vetor de entrada
        X_bias = np.r_[features, -1]

        # Sentido direto - cálculo da ativação e a saída de cada camada
        U = self.W @ X_bias         # Ativação de cada neurônio oculto
        Y = self.activation( U )    # Saída dos neurônios ocultos

        Z = np.r_[ Y, -1 ]          # Prepara as entradas para os neurônios de saída

        A = self.M @ Z              # Ativação dos neurônio de saída
        O = self.activation ( A )   # Saída da camada de saída

        # Monta um vetor de predição baseado no argmax da saída da rede
        predicted_output = -np.ones_like(O)
        predicted_output[ np.argmax(O) ] = +1

        # Retorna a classe correspondente ao vetor predito pela rede
        return self.train_dataset.decode_vector( predicted_output ) 

In [24]:
mlp = MLP( train_dataset )
mlp.train()

Época 0: 127 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 50: 38 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 100: 39 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 150: 31 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 200: 30 erros. - saída [-1. -1.  1.] [[-1. -1.  1.]]
Época 250: 42 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 300: 35 erros. - saída [ 1. -1. -1.] [[ 1. -1. -1.]]
Época 350: 40 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 400: 28 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 450: 31 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 500: 33 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 550: 30 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 600: 33 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 650: 30 erros. - saída [ 1. -1. -1.] [[-1. -1.  1.]]
Época 700: 33 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 750: 30 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 800: 30 erros. - saída [-1. -1.  1.] [[-1. -1.  1.]]

In [ ]:
q = 6                                   # Número de neurônios ocultos
m = train_dataset.class_count           # Número de neurônios de saída
p = train_dataset.features_count + 1    # Número de entradas da rede
eta = 0.1                               # Taxa de aprendizado

# Função de ativação usada
activation = lambda x: np.tanh(x)
ddx_activation = lambda x: 1 - activation(x) ** 2

# Inicializa o vetor de pesos dos neurônios ocultos
W = np.random.normal( size = (q, p) )

# Inicializa o vetor de pesos dos neurônios de saída
M = np.random.normal( size = (m, q+1) )

# Número de épocas de treinamento
max_epocas = 5000

# Percorre um número qualquer de épocas
for epoca in range( max_epocas ):
    total_erros = 0

    # Para cada época, embaralha o conjunto de treinamento
    shuffled_dataset = train_dataset.shuffle()

    # Percorre os exemplos do conjunto de treinamento
    for index, *features, classe in shuffled_dataset:
        # Vetor que representa a classe 
        real_output = train_dataset.encode_label( classe )

        # Monta o vetor de entrada
        X_bias = np.r_[features, -1]

        # Sentido direto - cálculo da ativação e a saída de cada camada
        U = W @ X_bias          # Ativação de cada neurônio oculto
        Y = activation( U )     # Saída dos neurônios ocultos

        Z = np.r_[ Y, -1 ]      # Prepara as entradas para os neurônios de saída

        A = M @ Z               # Ativação dos neurônio de saída
        O = activation ( A )    # Saída da camada de saída

        # Sentido inverso - atualização dos pesos das camadas
        err = real_output - O 

        # Verifica se houve erros
        if np.any(err != 0):
            # atualiza os pesos da camada de saída
            delta_output = err * ddx_activation( A )                        # (m x 1)
            delta_weights_out = eta * np.outer( delta_output, Z )           # (m x (q+1))
            M = M + delta_weights_out

            # calcula os erros retropagados para a camada oculta oculto
            output_weights_no_bias_T = M[:, :-1].T                          # (q x m)
            backpropagated_error = output_weights_no_bias_T @ delta_output  # (q x 1)

            # atualiza os pesos da camada oculta
            delta_hidden = backpropagated_error * ddx_activation(U)         # (q x 1)
            delta_weights_hidden = eta * np.outer( delta_hidden, X_bias )   # (q x p)
            W = W + delta_weights_hidden

        # Monta um vetor de predição baseado no argmax da saída da rede
        predicted_class = -np.ones_like(O)
        predicted_class[ np.argmax(O) ] = +1

        if np.argmax(O) != np.argmax(real_output):
            total_erros += 1

    if (epoca% round(max_epocas*0.05)) == 0:
        print(f"Época {epoca}: {total_erros} erros. - saída {predicted_class} [{real_output}]")
        

Época 0: 138 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]
Época 250: 34 erros. - saída [-1.  1. -1.] [[-1.  1. -1.]]


KeyboardInterrupt: 

In [ ]:
PS = PerceptronSimples( train_dataset )
PS.train( 500 )

for index, *point_test, classe in test_dataset:
    classe_prevista = PS.predict( point_test )
    print(f"{index}] Previu {classe_prevista} e era {classe} [{classe_prevista == classe}]")

train_dataset._centroids